# Analyse en Composantes Principales


L'Analyse en composantes principales est une méthode pour interpréter les données. Elle fonctionne sur des données décrites par par des attributs réels et est basée sur la méthodes des projections vue précédemment.

Cette méthode cherche à déterminer des tendances dans les données, et par là des oppositions grand/petit, gauche/droite, etc. Cette méthode est d'autant plus remarquable que ces tendances/oppositions peuvent être quantifiées : les interprétations (toujours subjectives) de nos nos données sont alors soutenus par des valeurs numériques (objectives).

Pour illustrer la méthode on utilisera nos (maintenant classiques) iris de fisher.

## Données : les iris de fisher

https://fr.wikipedia.org/wiki/Iris_de_Fisher On a pas besoin des espèces, on veut voir ce qu'il y a dans les données.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas

sns.set()

In [ ]:
data = sns.load_dataset('iris').drop(columns="species")
data

## Matrice X

Nous allons centrer et réduire les données.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pandas.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

In [ ]:
X

### Caractéristiques initiales 2 à 2

Nos données représentent 150 iris selon 4 caractéristiques. Commençons par représenter graphiquement les axes 2 à 2.

> **Attention** : [`seaborn.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html) n'est pas compatible avec les axes de matplotlib.

In [ ]:
sns.pairplot(X)

plt.show()

* La moyenne de chaque variable à 0 signifie que le centre de gravité du nuage est l'origine.
* L'écart type de chaque variable est 1 ce qui signifie que les valeurs vont être presque toutes comprisent en -2 et 2

Le données se répartissent selon des axes (les attributs).

Il arrive que certains axes permettent de déterminer des groupes dans nos données, par exemple ici `petal_length` qui a l'air de séparer nos iris en 2 groupes :

* un resserré sur les petites valeurs
* un autre plus long prenant les valeurs intermédiaire à grande

Mais la plupart du temps, comme par exemple ici `sepal_length`, les données s'étalent tout au long de l'axe. **chaque axe détermine une tendance au sein de nos données** : elle vont de petit à grand selon cet axe.


> Nos données étant réduite, **chaque axe explique 1/4 de l'information totale**.

#### Corrélation

La corrélation linéaire permet d'expliquer un axe par rapport à un autre. 

In [ ]:
X.corr()

On voit que :

* la longueur du pétale est très corrélée positivement à sa largeur
* la longueur du sépale est corrélée positivement à la largeur et à la longueur du pétale.

## But de l'analyse en composantes principale

Les axes déterminent des tendances au sein des données. La méthode cherche à déterminer les tendances les plus importantes (principales) des données.

**Attention** : on ne cherche pas à supprimer des axes initiaux ou a expliquer un axe initial par un autre. On cherche à expliquer nos données en considérant tous nos attributs.

En effet, si l'on a choisi de prendre tous ces attributs pour décrire nos données c'est que l'on pense qu'ils sont chacun aussi important l'un que l'autre (le choix des attributs a été une étape antérieure).


Pour cela, on cherche les axes maximisant la projection, ceci permettant d'expliquer les données par rapport aux axes.

### Projections d'inertie maximale

On cherche tous les axes maximisant l'inertie.

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
from sklearn.decomposition import PCA
import numpy as np

pca = PCA() 
pca.fit(X)

U = np.transpose(pca.components_) # vecteurs propres
I = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C = pandas.DataFrame(X @ U, index=X.index) # nouvelles coordonnées

corrélations = pandas.DataFrame([[C[facteur].corr(X[column]) for facteur in C] for column in X], index=X.columns)
cos2 = (C**2).div((X**2).sum(axis=1), axis='index')

### Inertie et information

### Inertie totale

On a coutume de représenter l'inertie par axe.

In [ ]:
I

Si les chiffres, c'est bien, une représentation graphique est plus explicite : 

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.plot(ax=ax)

plt.title("inertie expliquée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show()

L'inertie est bien décroissante, le 1er axe (axe d'index 0) expliquant plus de 70% de l'information, le second (d'index 1) seulement un peu plus de 20%.

A eux deux ils expliquent donc plus de 90% de l'information totale ! Pour montrer cela, on va représenter l'inertie cumulée.

In [ ]:
I.cumsum()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7)) 

I.cumsum().plot(ax=ax)

plt.title("inertie expliquée cumulée")
plt.xlabel("axes principaux")
plt.ylabel("% d'inertie expliquée")

plt.show()

Le nombre d'axe à garder pour l'analyse peut être déterminé de deux façon :

* le pourcentage minimale d'information que l'on veut conserver
* chercher un point d'inflexion (annulation de la dérivée seconde) à la courbe d'inertie : l'ajout d'un axe devient inneficace par rapport au gain d'information.

Dans notre cas, deux axes permettent d'expliquer à eux seuls 95% de l'information.

### Interprétation des axes

Nous avons décidé de garder les 2 premiers axes. regardons comment les expliquer, en dessinant le cercle des corrélations

In [ ]:
axe_x = 0
axe_y = 1

couleurs = [sns.color_palette()[3]] * len(corrélations)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = corrélations.plot.scatter(x=axe_x, y=axe_y, ax=ax, 
                                 color=couleurs)


for index, row in corrélations.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))

draw.add_patch(plt.Circle((0, 0), radius=1, color=(0, 0, 0, .3)))

plt.xlim(-1.2, 1.2)
plt.ylim(-1.2, 1.2)
plt.axvline(0)
plt.axhline(0)

plt.show()

Si l'on regarde les éléments les pls corrélés avec chaque axe :

* l'axe 0 (horizontal) s'explique par la longueur et largeur du pétale ainsi que la longueur du sépale
* l'axe 1 (vertical) s'explique par la largeur du sépale

Si l'on avait cherché à expliquer les axes 2 et 3 qui expliquent respectivement 3% et .5% de l'information, on pourrait regarder les cercles de corrélations pour ces deux axes. Nous n'allons pas le faire ici.

### Dessiner les projections

In [ ]:
C

Représentons nos iris sur les axes principaux 0 et 1

In [ ]:
axe_x = 0
axe_y = 1

couleurs = [sns.color_palette()[0]] * len(C)

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs , ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show() 

On n'y voit pas grand chose. Regroupons les iris par espèces. 

In [ ]:
# palette de couleurs

current_palette = sns.color_palette()
sns.palplot(current_palette)
plt.show() 

In [ ]:
# une couleur pour chaque classe d'iris

colors = [current_palette[0], current_palette[2], current_palette[3]]
iris_colors = [colors[i // 50] for i in range(len(X))]

In [ ]:
axe_x = 0
axe_y = 1

couleurs = iris_colors

fig, ax = plt.subplots(figsize=(7, 7)) 

draw = C.plot.scatter(axe_x, axe_y, color=couleurs, ax=ax)

for index, row in C.iterrows():
    draw.annotate(str(index), (row[axe_x], row[axe_y]))
    
plt.axvline(0)
plt.axhline(0)

plt.show() 

On remarque que les iris bleues sont bien représenté à gauche, puis il y a un gradiant :
* les bleus sont petites
* les verts sont moyennes
* les rouges sont grandes

Et c'est pas clair entre moyen et grand.

La dispersion selon cet axe explique 73% de nos données.

### Qualité de la projection

Globalement, les points vont bien se représenter sur l'axe, mais individuellement, il peut exister des points qui en sont éloignés. 

On calcule la qualité de la projection en calculant l'angle entre le point initial et le point projeté. Ceci parce que ça se calcule bien : le rapport entre la norme du projeté sur la norme du point dont le carré correspond au cosinus carré l'angle $\theta$ entre les 2 vecteur : $cos(\theta)^2 = \frac{||p(x_i)||^2}{||x_i||^2}$

Regardons les 10 iris qui se représentent le mieux sur le premier axe (cos2 très grand) :

In [ ]:
cos2[0].sort_values().tail(10)

Regardons les 10 iris qui se représentent le moins bien sur le premier axe (cos2 très petit) :

In [ ]:
cos2[0].sort_values().head(10)

En regardant les statistiques descriptives des $\cos^2$ pour le 1er axe :

In [ ]:
cos2[0].describe()

En gros ça se projette bien, 75% des points à un angle inférieur à 45°

In [ ]:
import math

math.degrees(math.acos(math.sqrt(0.488812)))

Angle en degrés pour les 2 premiers axes : 

In [ ]:
# angle: acos(sqrt(p))

from math import sqrt, acos, pi

cos2[[0, 1]].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi)

Les $\cos^2$ triées. On voit que l'angle maximum est de 63° :

In [ ]:
cos2[[0, 1]].sum(axis=1).apply(lambda x: acos(sqrt(x)) * 180 / pi).sort_values()